In [1]:
import numpy as np
import os
import open3d as o3d

In [2]:
def read_bin(path):
    #READING FROM FILE  
    dim=[0,0]
    fileData= []
    with open(path, 'rb') as f:
        frame = int.from_bytes(f.read(4),byteorder = 'little')
        dim[0] = int.from_bytes(f.read(4),byteorder = 'little')
        dim[1] = int.from_bytes(f.read(4),byteorder = 'little')
        byteData = f.read(4)
        while byteData:  # データがあるだけ以下を繰り返す
            n = int.from_bytes(byteData, byteorder='little')  # 読み込んだデータを整数に変換
            fileData.append(n)  # リストに追加
            byteData = f.read(4)

    #CONVERTING TO DEPTH MAP FORMAT 
    depth = list(map(float,fileData))
    depthCountPerMap = dim[0] * dim[1]
    depthMap = np.zeros((frame,dim[1],dim[0]),dtype=float)
    for i in range(frame):
        startIndex = i * depthCountPerMap
        endIndex = (i+1) * depthCountPerMap
        currentDepthData = np.array(depth[startIndex:endIndex])
        depthMap[i,:,:] = currentDepthData.reshape(dim[1],dim[0])
    return depthMap

In [3]:
def furtest_point_sampling(points, n_samples):
    """
    points: [N, 3] array containing the whole point cloud
    n_samples: samples you want in the sampled point cloud typically << N 
    """
    #initialize
    points = np.array(points)
    points_left = np.arange(len(points))
    sample_inds = np.zeros(n_samples, dtype='int')
    dists = np.ones_like(points_left) * float('inf')
    selected = 0
    sample_inds[0] = points_left[selected]
    points_left = np.delete(points_left, selected)

    for i in range(1, n_samples):
        last_added = sample_inds[i-1]    
        dist_to_last_added_point = (
            (points[last_added] - points[points_left])**2).sum(-1)
        dists[points_left] = np.minimum(dist_to_last_added_point, 
                                        dists[points_left])
        selected = np.argmax(dists[points_left])
        sample_inds[i] = points_left[selected]
        points_left = np.delete(points_left, selected)

    return points[sample_inds]

In [4]:
def concatenate_seq_pcd(a):
    pcd = a[0]
    for i in range(1,len(a)):
        pcd = np.concatenate([pcd,a[i]])
    return pcd

In [5]:
def process_one_action(path,ply=False):
    depth = read_bin(path)
    focal = 280
    xx, yy = np.meshgrid(np.arange(depth.shape[2]), np.arange(depth.shape[1]))

    point_clouds = []
    for d in range(depth.shape[0]):
        depth_map = depth[d]

        depth_min = depth_map[depth_map > 0].min()
        depth_map = depth_map

        x = xx[depth_map > 0]
        y = yy[depth_map > 0]
        z = depth_map[depth_map > 0]
        x = (x - depth_map.shape[1] / 2) / focal * z
        y = (y - depth_map.shape[0] / 2) / focal * z

        points = np.stack([x, y, z], axis=-1)

        point_clouds.append(points)

    if ply:
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(np.array(point_clouds[0]))
        o3d.io.write_point_cloud("DATA/MSRAction_npz/" + path.split('/')[2][:-4] + ".ply", pcd)
    else:
        pcd = concatenate_seq_pcd(point_clouds)
        pcd = furtest_point_sampling(pcd,1024)
        np.savez_compressed("DATA/MSRAction_npz_pp/" + path.split('/')[2][:-4] + '.npz', point_clouds=pcd)

In [6]:
def show_pointclouds(point_clouds):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(point_clouds)
    o3d.visualization.draw_geometries([pcd])

In [ ]:
import multiprocessing
from tqdm import tqdm

input_dir = 'DATA/MSRAction3D_bin'
files = os.listdir(input_dir)
pool = multiprocessing.Pool(4)

for input_file in files:
    process_one_action(input_dir +"/"+ input_file)
    pool.apply_async(process_one_action,(os.path.join(input_dir, input_file)))

pool.close()
pool.join()